# Get Side Information

In [ ]:
# run this code to get side information in floder 'info_data/'
import pandas as pd
import numpy as np
import tqdm
import datetime
import os
tqdm_ncols = 80
def updata_data(old_data, one_key, one_row_data):
    if one_key not in old_data.keys():
        old_data[one_key] = [
            one_row_data['is_right'], one_row_data['use_time'], 1
        ]
        return old_data, [0, 0, 0]
    else:
        one_data = old_data[one_key]
        re_data = [(one_data[0] / one_data[2]) * 100,
                   one_data[1] / one_data[2], one_data[2]]
        one_data[0] += one_row_data['is_right']
        one_data[1] += one_row_data['use_time']
        one_data[2] += 1
        old_data[one_key] = one_data
        return old_data, re_data


def updata_answer_data(old_data, one_key, one_row_data):
    if one_key not in old_data.keys():
        old_data[one_key] = [
            one_row_data['is_right'], one_row_data['use_time'], 1
        ]
        return old_data, [
            one_row_data['is_right'] * 100, one_row_data['use_time'], 1
        ]
    else:
        one_data = old_data[one_key]
        one_data[0] += one_row_data['is_right']
        one_data[1] += one_row_data['use_time']
        one_data[2] += 1
        old_data[one_key] = one_data
        re_data = [(one_data[0] / one_data[2]) * 100,
                   one_data[1] / one_data[2], one_data[2]]
        return old_data, re_data


def get_all_info(all_info, col_name, col_value, new_record_info):
    if col_value not in all_info.keys():
        one_q_data = new_record_info[new_record_info[col_name] == col_value]
        all_info[col_value] = [
            np.mean(one_q_data['is_right']),
            np.mean(one_q_data['use_time']),
            len(one_q_data.index)
        ]
    return all_info


def updata_time_data(old_data, one_key, create_time_now):
    if one_key not in old_data.keys():
        old_data[one_key] = create_time_now
        return old_data, 0
    else:
        old_time = old_data[one_key]
        re_data = create_time_now - old_time
        old_data[one_key] = create_time_now
        return old_data, re_data
    
def updata_passed_data(old_data, one_key, now_index):
    if one_key not in old_data.keys():
        old_data[one_key] = [now_index]
        return old_data, []
    else:
        re_data = old_data[one_key].copy()
        old_data[one_key].append(now_index)
        return old_data, re_data


def get_data(raw_data, stu_kp_data, stu_q_data, kp_data, q_data, q_all_data,
             kp_all_data, stu_kp_data_an, stu_q_data_an, kp_data_an,
             q_data_an,last_kp_time,last_q_time,passed_kp_list,passed_q_list):
    all_data = []
    for i in tqdm.tqdm(raw_data.index,
                       desc='Process',
                       mininterval=0.2,
                       ncols=tqdm_ncols):
        one_data = []
        one_re_data = raw_data.loc[i]
        stu_id = one_re_data['stu_id']
        kp_id = one_re_data['kp_id']
        q_id = one_re_data['q_id']
        stu_kp = str(int(stu_id)) + '_' + str(int(kp_id))
        one_time = one_re_data['create_time']

        stu_kp_data, one_stu_kp_data = updata_data(
            stu_kp_data,
            stu_kp, one_re_data)
        one_data.extend(one_stu_kp_data)

        # 'stu_q_now_acc', 'stu_q_now_time', 'stu_q_now_len'
        stu_q_data, one_stu_q_data = updata_data(stu_q_data, str(int(stu_id)),
                                                 one_re_data)
        one_data.extend(one_stu_q_data)

        # 'q_now_acc','q_now_use_time','q_now_len'
        q_data, one_q_data = updata_data(q_data, str(int(q_id)), one_re_data)
        one_data.extend(one_q_data)
        
        # 'kp_now_acc','kp_now_use_time','kp_now_len'
        kp_data, one_kp_data = updata_data(kp_data, str(int(kp_id)),
                                           one_re_data)
        one_data.extend(one_kp_data)

        # 'q_all_acc','q_all_use_time','q_all_len'
        q_all_data = get_all_info(q_all_data, 'q_id', q_id, raw_data)
        one_data.extend(q_all_data[q_id])

        # 'kp_all_acc','kp_all_use_time','kp_all_len'
        kp_all_data = get_all_info(kp_all_data, 'kp_id', kp_id, raw_data)
        one_data.extend(kp_all_data[kp_id])

        # 'stu_kp_now_acc_an','stu_kp_now_use_time_an','stu_kp_now_len_an'
        stu_kp_data_an, one_stu_kp_data_an = updata_answer_data(
            stu_kp_data_an,
            str(int(stu_id)) + '_' + str(int(kp_id)), one_re_data)
        one_data.extend(one_stu_kp_data_an)

        # 'stu_q_now_acc_an', 'stu_q_now_time_an', 'stu_q_now_len_an'
        stu_q_data_an, one_stu_q_data_an = updata_answer_data(
            stu_q_data_an, str(int(stu_id)), one_re_data)
        one_data.extend(one_stu_q_data_an)

        # 'q_now_acc_an','q_now_use_time_an','q_now_len_an'
        q_data_an, one_q_data_an = updata_answer_data(q_data_an,
                                                      str(int(q_id)),
                                                      one_re_data)
        one_data.extend(one_q_data_an)
        # 'kp_now_acc_an','kp_now_use_time_an','kp_now_len_an'
        kp_data_an, one_kp_data_an = updata_answer_data(
            kp_data_an, str(int(kp_id)), one_re_data)
        one_data.extend(one_kp_data_an)
        
        # 'last_kp_time_passed','last_q_time_passed'
        last_kp_time,last_kp_time_passed = updata_time_data(last_kp_time,stu_kp, one_time)
        last_q_time,last_q_time_passed = updata_time_data(last_q_time,str(int(stu_id)),one_time)
        one_data.extend([last_kp_time_passed,last_q_time_passed])
        
        # 'passed_kp','passed_q'
        passed_kp_list,passed_kp = updata_passed_data(passed_kp_list,stu_kp,i)
        passed_q_list,passed_q = updata_passed_data(passed_q_list,str(int(stu_id)),i)
        one_data.extend([passed_kp,passed_q])
        
        all_data.append(one_data)
    all_data_2 = np.array(all_data,dtype=object)
    new_col = [
        'stu_kp_now_acc', 'stu_kp_now_use_time', 'stu_kp_now_len',
        'stu_q_now_acc', 'stu_q_now_time', 'stu_q_now_len', 
        'q_now_acc', 'q_now_use_time', 'q_now_len', 
        'kp_now_acc', 'kp_now_use_time','kp_now_len', 
        'q_all_acc', 'q_all_use_time', 'q_all_len', 
        'kp_all_acc','kp_all_use_time', 'kp_all_len', 
        'stu_kp_now_acc_an','stu_kp_now_use_time_an', 'stu_kp_now_len_an', 
        'stu_q_now_acc_an','stu_q_now_time_an', 'stu_q_now_len_an', 
        'q_now_acc_an','q_now_use_time_an', 'q_now_len_an', 
        'kp_now_acc_an','kp_now_use_time_an', 'kp_now_len_an',
        'last_kp_time_passed','last_q_time_passed','passed_kp','passed_q'
    ]
    processed_data = raw_data.copy()
    for j, one_col in enumerate(new_col):
        processed_data[one_col] = all_data_2[:, j]
    return processed_data

# Side information extraction
all_dataset = [ 'ASSIST2009NSB', 'ASSIST2012', 'EdNet']
for one_dataset in all_dataset:
    data_name = one_dataset
    print(f'{data_name} processing')
    raw_data = pd.read_pickle(f'raw_data/{data_name}.pkl')
    print(raw_data.head())
    if data_name in ['ASSIST2012']:
        raw_data['create_time'] = pd.to_datetime(raw_data['create_time'])
        raw_data['create_time'] =pd.to_timedelta(raw_data['create_time']-raw_data['create_time'][0]).dt.total_seconds()
    stu_kp_data = {}
    stu_q_data = {}
    kp_data = {}
    q_data = {}
    q_all_data = {}
    kp_all_data = {}
    stu_kp_data_an = {}
    stu_q_data_an = {}
    kp_data_an = {}
    q_data_an = {}
    last_kp_time = {}
    last_q_time = {}
    passed_kp_list = {}
    passed_q_list = {}
    processed_data= get_data(raw_data, stu_kp_data, stu_q_data, kp_data, q_data, q_all_data,kp_all_data, stu_kp_data_an, stu_q_data_an, kp_data_an, q_data_an,last_kp_time,last_q_time,passed_kp_list,passed_q_list)
    processed_data_name = f'info_data/{data_name}_info.pkl'
    processed_data.to_pickle(processed_data_name)
    print(data_name, 'Side information extraction processing completed！file at：', processed_data_name)
    processed_data.head()

# Get Sequence Data

In [ ]:
# run this code to get sequence data in floder 'final_data/'
import tqdm
import numpy as np
import datetime
import time
import math
import pandas as pd
from sklearn.preprocessing import StandardScaler
old_col = [
    'stu_id', 'seq_id', 'kp_id', 'q_id', 'create_time', 'use_time', 'is_right',
    'stu_kp_now_acc', 'stu_kp_now_use_time', 'stu_kp_now_len', 'stu_q_now_acc',
    'stu_q_now_time', 'stu_q_now_len', 'q_now_acc', 'q_now_use_time',
    'q_now_len', 'kp_now_acc', 'kp_now_use_time', 'kp_now_len', 'q_all_acc',
    'q_all_use_time', 'q_all_len', 'kp_all_acc', 'kp_all_use_time',
    'kp_all_len', 'stu_kp_now_acc_an', 'stu_kp_now_use_time_an',
    'stu_kp_now_len_an', 'stu_q_now_acc_an', 'stu_q_now_time_an',
    'stu_q_now_len_an', 'q_now_acc_an', 'q_now_use_time_an', 'q_now_len_an',
    'kp_now_acc_an', 'kp_now_use_time_an', 'kp_now_len_an',
    'last_kp_time_passed', 'last_q_time_passed', 'passed_kp', 'passed_q'
]

select_col = [
    'stu_kp_now_acc',
    'stu_kp_now_use_time',
    'stu_kp_now_len',
    'stu_q_now_acc',
    'stu_q_now_time',
    'stu_q_now_len',
    'q_now_acc',
    'q_now_use_time',
    'q_now_len',
    'kp_now_acc',
    'kp_now_use_time',
    'kp_now_len',
    'last_kp_time_passed',
    'last_q_time_passed',
    'stu_kp_now_acc_an',
    'stu_kp_now_use_time_an',
    'stu_kp_now_len_an',
    'stu_q_now_acc_an',
    'stu_q_now_time_an',
    'stu_q_now_len_an',
    'q_now_acc_an',
    'q_now_use_time_an',
    'q_now_len_an',
    'kp_now_acc_an',
    'kp_now_use_time_an',
    'kp_now_len_an',
    'use_time'
]
ITEM_COL = ['stu_kp_now_acc', 'stu_kp_now_use_time', 'stu_kp_now_len', 'stu_q_now_acc',
            'stu_q_now_time', 'stu_q_now_len', 'q_now_acc', 'q_now_use_time',
            'q_now_len', 'kp_now_acc', 'kp_now_use_time', 'kp_now_len',
            'last_kp_time_passed', 'last_q_time_passed']
ITEM_AN_COL = ['stu_kp_now_acc_an', 'stu_kp_now_use_time_an', 
               'stu_kp_now_len_an', 'stu_q_now_acc_an', 
               'stu_q_now_time_an', 'stu_q_now_len_an',
               'q_now_acc_an', 'q_now_use_time_an', 
               'q_now_len_an', 'kp_now_acc_an', 
               'kp_now_use_time_an', 'kp_now_len_an', 
               'last_kp_time_passed', 'last_q_time_passed',
               'use_time', 'is_right']

def standard_scaler(DF):
    st_sc = StandardScaler()
    scaler_data_x = st_sc.fit_transform(DF)
    scaler_data_ed = pd.DataFrame(scaler_data_x, columns=DF.columns)
    return scaler_data_ed
def get_memory(item_list):
    re_data = np.zeros(shape=[MEMORY_LEN, len(ITEM_AN_COL)+1])
    len_i = len(item_list)
    if len_i != 0:
        get_list = item_list[-MEMORY_LEN:]
        for i, one in enumerate(get_list):
            one_data = list(RAW_DATA.loc[one][ITEM_AN_COL])
            # add mask
            one_data.append(1)
            re_data[i,:] = one_data
    seq_data = re_data[:, :-1]
    padding_mask = re_data[:, -1]
    if sum(padding_mask) == 0:
        padding_mask[0] = 1
    return seq_data, padding_mask
MEMORY_LEN = 5
all_dataset = [ 'ASSIST2009NSB', 'ASSIST2012', 'EdNet']
for one_dataset in all_dataset:
    data_name = one_dataset
    print(f'{data_name} processing')
    processed_data = pd.read_pickle(f'info_data/{data_name}_info.pkl')
    scaler_data = processed_data[select_col]
    scaler_data = standard_scaler(scaler_data)
    other_data = processed_data[['is_right','passed_kp', 'passed_q']]
    RAW_DATA = scaler_data.join(other_data)
    del processed_data
    del scaler_data
    del other_data
    all_data = list(RAW_DATA.index)
    all_stu_kp_seq = []
    all_stu_kp_seq_padding_mask = []
    all_stu_q_seq = []
    all_stu_q_seq_padding_mask = []
    for raw_index in tqdm.tqdm(all_data, desc='Process', mininterval=0.2,ncols=80):
        one_data = RAW_DATA.loc[raw_index]
        stu_kp_seq, stu_kp_seq_padding_mask = get_memory(
            list(one_data['passed_kp']))
        stu_q_seq, stu_q_seq_padding_mask = get_memory(
            list(one_data['passed_q']))
        all_stu_kp_seq.append(stu_kp_seq)
        all_stu_kp_seq_padding_mask.append(stu_kp_seq_padding_mask)
        all_stu_q_seq.append(stu_q_seq)
        all_stu_q_seq_padding_mask.append(stu_q_seq_padding_mask)
    RAW_DATA['stu_kp_seq'] = all_stu_kp_seq
    del all_stu_kp_seq
    RAW_DATA['stu_kp_seq_padding_mask'] = all_stu_kp_seq_padding_mask
    del all_stu_kp_seq_padding_mask
    RAW_DATA['stu_q_seq'] = all_stu_q_seq
    del all_stu_q_seq
    RAW_DATA['stu_q_seq_padding_mask'] = all_stu_q_seq_padding_mask
    del all_stu_q_seq_padding_mask
    file_name = f'final_data/{data_name}_final_{str(MEMORY_LEN)}.pkl'
    RAW_DATA.to_pickle(file_name)
    print(data_name, 'sequence data extraction processing completed！file at：', file_name)
    RAW_DATA.head()
    del RAW_DATA

# Partition Dataset

In [ ]:
# Dividing datasets to train data and test data
import pandas as pd
from sklearn.model_selection import train_test_split
all_dataset = [ 'ASSIST2009NSB', 'ASSIST2012', 'EdNet']
for one_dataset in all_dataset:
    DATA_NAME = one_dataset
    processed_data = pd.read_pickle(f'info_data/{DATA_NAME}_info.pkl')
    all_stu = processed_data['stu_id'].value_counts().index
    print(DATA_NAME,'number of students:',len(all_stu))
    test_size = 3000
    if DATA_NAME == 'EdNet':
        test_size = 50000
    X_test, X_train = train_test_split(all_stu, random_state=2022, test_size=test_size, shuffle=True)
    select_stu = X_train
    trian_data = processed_data[processed_data['stu_id'].isin(select_stu)]
    trian_index = trian_data.index
    test_data = processed_data[processed_data['stu_id'].isin(select_stu)==0]
    test_index = test_data.index
    print(DATA_NAME,'train data length:',len(trian_index))
    print(DATA_NAME,'test data length:',len(test_index))
    MEMORY_LEN = 5
    RAW_DATA = pd.read_pickle(f'../DataProcess/final_data/{DATA_NAME}_final_{str(MEMORY_LEN)}.pkl')
    one_test_data = RAW_DATA.loc[test_index]
    one_test_data.to_pickle(f'../DataProcess/test_data/{DATA_NAME}_test_{str(MEMORY_LEN)}.pkl')
    one_train_data = RAW_DATA.loc[trian_index]
    one_train_data.to_pickle(f'../DataProcess/train_data/{DATA_NAME}_final_{str(MEMORY_LEN)}.pkl')